In [88]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf 

In [61]:
cars = pd.read_csv('cars.csv')

In [62]:
cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [63]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [64]:
cars.shape

(205, 26)

In [65]:
cars['company'] = cars.CarName.apply(lambda x: x.replace('-', ' ').split()[0])

In [66]:
cars['company'].nunique()

28

In [67]:
cars = cars
cars.drop(columns=['CarName', 'car_ID'], inplace=True)

In [68]:
def corrector(x):
    if x in ('Nissan', 'nissan'):
        return 'nissan'
    elif x in ('vokswagen', 'vw', 'volkswagen'):
        return 'volkswagen'
    elif x in ('maxda', 'mazda'):
        return 'mazda'
    elif x in ('porcshce', 'porsche'):
        return 'porsche'
    elif x in ('toyouta', 'toyota'):
        return 'toyota'
    else:
        return x

In [69]:
cars['company'] = cars.company.apply(corrector)

In [70]:
cars['company'].nunique()

22

In [71]:
cars = cars[['company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower', 'price']]

In [73]:
cars.corr()

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price
wheelbase,1.000000,0.874587,0.795144,0.776386,0.569329,0.488750,0.353294,0.577816
carlength,0.874587,1.000000,0.841118,0.877728,0.683360,0.606454,0.552623,0.682920
carwidth,0.795144,0.841118,1.000000,0.867032,0.735433,0.559150,0.640732,0.759325
curbweight,0.776386,0.877728,0.867032,1.000000,0.850594,0.648480,0.750739,0.835305
enginesize,0.569329,0.683360,0.735433,0.850594,1.000000,0.583774,0.809769,0.874145
boreratio,0.488750,0.606454,0.559150,0.648480,0.583774,1.000000,0.573677,0.553173
horsepower,0.353294,0.552623,0.640732,0.750739,0.809769,0.573677,1.000000,0.808139
price,0.577816,0.682920,0.759325,0.835305,0.874145,0.553173,0.808139,1.000000


In [79]:
cars.dtypes == 'object'

company            True
fueltype           True
aspiration         True
carbody            True
drivewheel         True
wheelbase         False
carlength         False
carwidth          False
curbweight        False
enginetype         True
cylindernumber     True
enginesize        False
boreratio         False
horsepower        False
price             False
dtype: bool

In [81]:
objects = cars.select_dtypes(['object']).columns

In [83]:
objects

Index(['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel',
       'enginetype', 'cylindernumber'],
      dtype='object')

In [84]:
cars[objects]

,company,fueltype,aspiration,carbody,drivewheel,enginetype,cylindernumber
0,alfa,gas,std,convertible,rwd,dohc,four
1,alfa,gas,std,convertible,rwd,dohc,four
2,alfa,gas,std,hatchback,rwd,ohcv,six
3,audi,gas,std,sedan,fwd,ohc,four
4,audi,gas,std,sedan,4wd,ohc,five
...,...,...,...,...,...,...,...
200,volvo,gas,std,sedan,rwd,ohc,four
201,volvo,gas,turbo,sedan,rwd,ohc,four
202,volvo,gas,std,sedan,rwd,ohcv,six
203,volvo,diesel,turbo,sedan,rwd,ohc,six


In [87]:
cars = pd.get_dummies(data=cars, columns=objects, drop_first = True)

In [98]:
cars.head()

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,88.6,168.8,64.1,2548,130,3.47,111,13495.0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,88.6,168.8,64.1,2548,130,3.47,111,16500.0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,94.5,171.2,65.5,2823,152,2.68,154,16500.0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,99.8,176.6,66.2,2337,109,3.19,102,13950.0,1,0,...,1,0,0,0,0,1,0,0,0,0
4,99.4,176.6,66.4,2824,136,3.19,115,17450.0,1,0,...,1,0,0,0,1,0,0,0,0,0


In [99]:
X = cars.drop(columns=['price'])
Y = cars.price

In [102]:
X1 = sm.add_constant(X.horsepower)  # добавить константу, чтобы был свободный член
model = sm.OLS(Y, X1)  # говорим модели, что у нас ЗП, а что НП
results = model.fit()  # строим регрессионную прямую
print(results.summary())  # смотрим результат

# способ второй, потенциально более удобный

# results = smf.ols('Y ~ X1 + X2 + ... + Xn', data).fit()
# print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Sat, 18 Nov 2023   Prob (F-statistic):           1.48e-48
Time:                        20:52:26   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3721.7615    929.849     -4.003      0.0

In [106]:
X1 = sm.add_constant(X)  # добавить константу, чтобы был свободный член
model = sm.OLS(Y, X1)  # говорим модели, что у нас ЗП, а что НП
results = model.fit()  # строим регрессионную прямую
print(results.summary())  # смотрим результат

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Sat, 18 Nov 2023   Prob (F-statistic):           4.86e-89
Time:                        20:54:22   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.472e+

In [109]:
companies = []
for i in X.columns:
    if i.startswith('company'):
        companies.append(i)
companies

['company_audi',
 'company_bmw',
 'company_buick',
 'company_chevrolet',
 'company_dodge',
 'company_honda',
 'company_isuzu',
 'company_jaguar',
 'company_mazda',
 'company_mercury',
 'company_mitsubishi',
 'company_nissan',
 'company_peugeot',
 'company_plymouth',
 'company_porsche',
 'company_renault',
 'company_saab',
 'company_subaru',
 'company_toyota',
 'company_volkswagen',
 'company_volvo']

In [110]:
X1 = sm.add_constant(X.drop(columns=companies))  # добавить константу, чтобы был свободный член
model = sm.OLS(Y, X1)  # говорим модели, что у нас ЗП, а что НП
results = model.fit()  # строим регрессионную прямую
print(results.summary())  # смотрим результат


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Sat, 18 Nov 2023   Prob (F-statistic):           9.86e-81
Time:                        20:57:44   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -1.7e+